<a href="https://colab.research.google.com/github/amisaid/ML-Zoomwork-Homework/blob/main/fcc_predict_health_costs_with_regression_050323.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

### Assessing the dataset

In [ ]:
## Checking null values in the dataframe
dataset.isnull().sum()

In [ ]:
# Statistical summary of the data 
dataset.describe()

In [ ]:
# Checking the data formats
dataset.dtypes

In [ ]:
# Setting the numerical rows into a list
numerical_ds = ["age", "bmi", "children", "expenses"]
categorical_ds = ["sex", "smoker", "region"]

numerical_ds, categorical_ds

In [ ]:
#Checking unique values in the dataframe
for col in dataset.columns:
    print(col)
    print(dataset[col].unique()[:5])
    print(dataset[col].nunique())
    print()


### Task 1: 
Converting categorical to numerical and splitting dataset into 80% for training and 20% for testing

In [ ]:
categorical = {}

for c in categorical_ds:
    categorical[c] = list(dataset[c].value_counts().head().index)

# categorical

# Converting categorical data to numerical 
for name, values in categorical.items():
        for value in values:
            dataset['%s_%s' % (name, value)] = (dataset[name] == value).astype(int)
            numerical_ds.append('%s_%s' % (name, value))


df_full = dataset[numerical_ds]
df_full.head()

In [ ]:
df_full.columns

**Splitting the data into 80% training and 20% for testing**

In [ ]:
# Checking the length of the dataset
n = len(df_full)
n

In [ ]:
n_test = int(n * 0.2)
n_train = n - n_test

# Verifying the sums tally
n_train, n_test, (n_train + n_test)


In [ ]:
train_dataset = df_full.iloc[:n_train]
test_dataset = df_full.iloc[n_train:]


### Task 2:

Creating labels by `pop` of "expenses" column 

In [ ]:
# Creating labels by "pop" of "expense" column
train_labels = train_dataset.pop("expenses")
test_labels = test_dataset.pop("expenses")

# Confirming pop is dropped from test_dataset
test_dataset.columns

### Task 3:

Creating a model and training using the `train_dataset` data.

In [ ]:
normalizer = tf.keras.layers.Normalization(axis=-1)

In [ ]:
model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=1)
])

In [ ]:
# Configure the model with Keras Model.compile and 
# train with Model.fit for 100 epochs:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.98),
    loss='mae',
    metrics=['mae', 'mse'])

In [ ]:
%%time
history = model.fit(
    train_dataset,
    train_labels,
    epochs=2000,
    # Suppress logging.
    verbose=0)

In [ ]:
test_results = {}

test_results['model'] = model.evaluate(
    test_dataset, test_labels, verbose=0)

In [ ]:
test_results

### Final Task:

Assessing the accuracy of the trained model

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)


In [ ]:
model.predict(train_dataset[:6])

In [ ]:
train_labels[:6]